## Загрузим нужные библиотеки

In [1]:
!pip install lightgbm
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.decomposition import PCA
from sklearn.svm import LinearSVC
from sklearn.linear_model import Perceptron
from sklearn.linear_model import RidgeClassifierCV
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import ExtraTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import recall_score
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV
from sklearn.svm import LinearSVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multiclass import OneVsOneClassifier
from sklearn.multiclass import OutputCodeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neighbors import RadiusNeighborsClassifier
from imblearn.over_sampling import SMOTE
import numpy as np
from sklearn.neighbors import NearestCentroid
import tensorflow as tf 
from tensorflow import keras
%matplotlib inline

2022-10-26 19:18:29.855122: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-26 19:18:29.978047: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-26 19:18:29.978067: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-10-26 19:18:30.012586: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-26 19:18:30.610639: W tensorflow/stream_executor/platform/de

В начале остановимся на наиболее перспективных навыках

In [2]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

df_calls = pd.read_csv("Calls.csv")
df_connection_time = pd.read_csv("ConnectionTime.csv")
df_ed = pd.read_csv("Education.csv")
df_tasks=pd.read_csv('Tasks.csv')
timennetwork=pd.read_csv('TimenNetwork.csv')
workingday=pd.read_csv("WorkingDay.csv")

/tmp/ipykernel_33784/1620961415.py:5: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_connection_time = pd.read_csv("ConnectionTime.csv")
/tmp/ipykernel_33784/1620961415.py:7: DtypeWarning: Columns (1,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_tasks=pd.read_csv('Tasks.csv')


In [3]:
df_train['type'].value_counts()

2    298
0    153
1     74
3     25
Name: type, dtype: int64

## Рассмотрим датасет по ближе

In [4]:
df_calls.head(3)

,Date,CallTime,NumberOfCalls,Вид учета времени,InOut,id
0,"2021-08-16 00:00:00,000","0,00027777777777777778",1,Будни,ToUser,ОРГ1-01945
1,"2021-09-21 00:00:00,000","0,00027777777777777778",1,Будни,ToUser,ОРГ1-01945
2,"2021-01-11 00:00:00,000","0,00027777777777777778",1,Будни,ToUser,ОРГ1-01945


In [5]:
df_calls['Date'].min()

'2021-01-01 00:00:00,000'

In [6]:
df_calls['Date'].max()

'2021-12-30 00:00:00,000'

In [7]:
df_connection_time.head(3)

,dateNum,maxLogOff,Нормативное время начала раб.дня,Фактич. время начала раб.дня,Время опоздания,Признак опоздания,Вых/Будни,id
0,"2021-12-15 00:00:00,000","2021-12-15 17:30:27,246","1899-12-30 08:30:00,000","1899-12-30 08:24:18,606",NaN,NaN,Будни,ОРГ1-02782
1,"2021-12-15 00:00:00,000","2021-12-15 16:30:13,330","1899-12-30 08:30:00,000","1899-12-30 01:05:20,513",NaN,NaN,Будни,ОРГ1-01407
2,"2021-12-15 00:00:00,000","2021-12-15 15:27:37,246","1899-12-30 08:30:00,000","1899-12-30 05:47:33,156",NaN,NaN,Будни,ОРГ1-01909


In [8]:
df_connection_time.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230662 entries, 0 to 230661
Data columns (total 8 columns):
 #   Column                            Non-Null Count   Dtype 
---  ------                            --------------   ----- 
 0   dateNum                           230662 non-null  object
 1   maxLogOff                         230662 non-null  object
 2   Нормативное время начала раб.дня  230662 non-null  object
 3   Фактич. время начала раб.дня      230662 non-null  object
 4   Время опоздания                   103101 non-null  object
 5   Признак опоздания                 77520 non-null   object
 6   Вых/Будни                         230662 non-null  object
 7   id                                230662 non-null  object
dtypes: object(8)
memory usage: 14.1+ MB


In [9]:
df_ed.head(3)

,Табельный номер руководителя,id,Вид образования,Специальность
0,NaN,ОРГ1-00131,Высшее образование - бакалавриат,Юриспруденция
1,NaN,ОРГ1-03220,Среднее профессиональное образование,Финансы
2,NaN,ОРГ1-03008,Высшее образование,Юриспруденция


In [10]:
df_train.head(3)

,id,type
0,ОРГ1-02050,2
1,ОРГ1-02783,2
2,ОРГ2-06173,0


In [11]:
df_test.head(3)

,id,type
0,ОРГ1-02649,0
1,ОРГ2-05929,0
2,ОРГ2-05859,0


Узнаем размерность наборов

In [12]:
print("df_train", df_train.shape)
print("df_calls", df_calls.shape)
print("df_connection_time", df_connection_time.shape)
print("df_ed", df_ed.shape)
print("df_test", df_test.shape)

df_train (550, 2)
df_calls (407867, 6)
df_connection_time (230662, 8)
df_ed (5609, 4)
df_test (262, 2)


### Видоизмение данных

Обьединим два набора с данными

In [13]:
df_train = pd.merge(df_train, df_ed , on="id", how='left').drop_duplicates(subset="id")

df_train.head(3)

,id,type,Табельный номер руководителя,Вид образования,Специальность
0,ОРГ1-02050,2,NaN,Неполное высшее образование,Технологические машины и оборудование
2,ОРГ1-02783,2,ОРГ1-01974,Высшее образование,Автоматизация технологических процессов и прои...
3,ОРГ2-06173,0,ОРГ2-01170,Высшее образование,Управление качеством


In [14]:
df_train = df_train.drop(["Табельный номер руководителя", "Специальность"],axis = 1)

df_train["Вид образования"].value_counts()

Высшее образование                                297
Среднее профессиональное образование               64
Высшее образование - бакалавриат                   59
Высшее образование - специалитет, магистратура     30
Начальное профессиональное образование              7
Повышение квалификации                              7
Неполное высшее образование                         5
Среднее общее образование                           3
Среднее (полное) общее образование                  2
Переподготовка                                      2
Профессиональное обучение                           1
Name: Вид образования, dtype: int64

In [15]:
df_train["Вид образования"] = df_train["Вид образования"].astype('category')
df_train["Вид образования"] = df_train["Вид образования"].cat.codes

df_train["Вид образования"].value_counts()

 0     297
-1      73
 10     64
 1      59
 2      30
 3       7
 6       7
 4       5
 9       3
 8       2
 5       2
 7       1
Name: Вид образования, dtype: int64

Оставим только столбец - опоздания сотрудников

In [16]:
df_connection_time_res = df_connection_time[["id","Признак опоздания"]]
df_connection_time_res = df_connection_time_res[df_connection_time_res["Признак опоздания"].notna()]

df_connection_time_res.head()

,id,Признак опоздания
127561,ОРГ1-01589,Опоздание
127562,ОРГ1-01589,Опоздание
127563,ОРГ1-01589,Опоздание
127564,ОРГ1-01589,Опоздание
127565,ОРГ1-01589,Опоздание


In [17]:
df_connection_time_res = df_connection_time_res.groupby("id").count()
df_connection_time_res = df_connection_time_res.rename(columns={"Признак опоздания": "Число опозданий"})

df_connection_time_res.head(3)

,Число опозданий
id,
ОРГ1-00004,275
ОРГ1-00028,35
ОРГ1-00030,57


In [18]:
df_train = pd.merge(df_train, df_connection_time_res , on="id", how='left')#.drop(["event_type"],axis =1)

df_train.head(3)

,id,type,Вид образования,Число опозданий
0,ОРГ1-02050,2,4,5.0
1,ОРГ1-02783,2,0,7.0
2,ОРГ2-06173,0,0,22.0


In [19]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 550 entries, 0 to 549
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               550 non-null    object 
 1   type             550 non-null    int64  
 2   Вид образования  550 non-null    int8   
 3   Число опозданий  491 non-null    float64
dtypes: float64(1), int64(1), int8(1), object(1)
memory usage: 17.7+ KB


## Добавим суммарное время звонков!



In [20]:
def str_to_float(str):
    if type(str)==float:
        return str
    str=str.replace(',','')
    return float(str)

df_calls['CallTime']=df_calls['CallTime'].map(str_to_float)
CallTime=pd.DataFrame(df_calls.groupby('id')['CallTime'].sum()).reset_index()
df_train = pd.merge(df_train, CallTime , on="id", how='left')
df_train

,id,type,Вид образования,Число опозданий,CallTime
0,ОРГ1-02050,2,4,5.0,4.810983e+18
1,ОРГ1-02783,2,0,7.0,1.350599e+19
2,ОРГ2-06173,0,0,22.0,NaN
3,ОРГ2-05359,2,10,6.0,NaN
4,ОРГ1-00803,2,0,35.0,1.388889e+17
...,...,...,...,...,...
545,ОРГ1-02587,2,-1,NaN,NaN
546,ОРГ1-02383,2,0,54.0,2.259805e+19
547,ОРГ1-02862,2,-1,NaN,NaN
548,ОРГ1-00471,2,-1,12.0,1.749719e+19


In [21]:
df_calls['NumberOfCalls'].value_counts()

1      140167
2       60780
3       33555
4       27493
5       19761
        ...  
107         1
105         1
114         1
98          1
95          1
Name: NumberOfCalls, Length: 102, dtype: int64

## Добавим колличество звонков в будни и выходные

In [22]:
NumberOfCalls=pd.DataFrame(df_calls.groupby('id')['NumberOfCalls'].sum()).reset_index()
df_calls= df_calls.rename(columns={'Вид учета времени': 'statusworkfre'})

NumberOfCallsW=pd.DataFrame(df_calls.query("statusworkfre=='Будни'").groupby('id')['NumberOfCalls'].sum()).reset_index()
NumberOfCallsNW=pd.DataFrame(df_calls.query("statusworkfre=='Выходные дни'").groupby('id')['NumberOfCalls'].sum()).reset_index()
df_train = pd.merge(df_train, NumberOfCalls , on="id", how='left')
df_train = pd.merge(df_train, NumberOfCallsW , on="id", how='left')
df_train = pd.merge(df_train, NumberOfCallsNW , on="id", how='left')
df_train = df_train.rename(columns={'NumberOfCalls_x': 'NumberOfCalls_worck','NumberOfCalls_y': 'NumberOfCalls_free','NumberOfCalls': 'NumberOfCallsfreesum'})
df_train


,id,type,Вид образования,Число опозданий,CallTime,NumberOfCalls_worck,NumberOfCalls_free,NumberOfCallsfreesum
0,ОРГ1-02050,2,4,5.0,4.810983e+18,546.0,546.0,NaN
1,ОРГ1-02783,2,0,7.0,1.350599e+19,1902.0,1902.0,NaN
2,ОРГ2-06173,0,0,22.0,NaN,NaN,NaN,NaN
3,ОРГ2-05359,2,10,6.0,NaN,NaN,NaN,NaN
4,ОРГ1-00803,2,0,35.0,1.388889e+17,7.0,7.0,NaN
...,...,...,...,...,...,...,...,...
545,ОРГ1-02587,2,-1,NaN,NaN,NaN,NaN,NaN
546,ОРГ1-02383,2,0,54.0,2.259805e+19,4570.0,4234.0,336.0
547,ОРГ1-02862,2,-1,NaN,NaN,NaN,NaN,NaN
548,ОРГ1-00471,2,-1,12.0,1.749719e+19,3917.0,3638.0,279.0


## Добавим колличество входящих и исходящих.

In [23]:
NumberToUser=pd.DataFrame(df_calls.query("InOut=='ToUser'").groupby('id')['NumberOfCalls'].sum()).reset_index()
NumberFromUser=pd.DataFrame(df_calls.query("InOut=='FromUser'").groupby('id')['NumberOfCalls'].sum()).reset_index() 
df_train = pd.merge(df_train, NumberToUser , on="id", how='left')
df_train = pd.merge(df_train, NumberFromUser , on="id", how='left')

## Добавим суммарное время опазданий

In [24]:
df_connection_time['Время опоздания']=df_connection_time['Время опоздания'].map(str_to_float)
falout=pd.DataFrame(df_connection_time.groupby('id')['Время опоздания'].sum()).reset_index() 
df_train = pd.merge(df_train, falout , on="id", how='left')
df_train

,id,type,Вид образования,Число опозданий,CallTime,NumberOfCalls_worck,NumberOfCalls_free,NumberOfCallsfreesum,NumberOfCalls_x,NumberOfCalls_y,Время опоздания
0,ОРГ1-02050,2,4,5.0,4.810983e+18,546.0,546.0,NaN,176.0,370.0,2.361381e+17
1,ОРГ1-02783,2,0,7.0,1.350599e+19,1902.0,1902.0,NaN,657.0,1245.0,2.532397e+17
2,ОРГ2-06173,0,0,22.0,NaN,NaN,NaN,NaN,NaN,NaN,1.200253e+18
3,ОРГ2-05359,2,10,6.0,NaN,NaN,NaN,NaN,NaN,NaN,1.657396e+17
4,ОРГ1-00803,2,0,35.0,1.388889e+17,7.0,7.0,NaN,1.0,6.0,9.288578e+17
...,...,...,...,...,...,...,...,...,...,...,...
545,ОРГ1-02587,2,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
546,ОРГ1-02383,2,0,54.0,2.259805e+19,4570.0,4234.0,336.0,1692.0,2878.0,2.432262e+18
547,ОРГ1-02862,2,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
548,ОРГ1-00471,2,-1,12.0,1.749719e+19,3917.0,3638.0,279.0,1088.0,2829.0,5.449973e+17


## Добавим дельту прихода на работу

In [25]:
df_connection_time['Нормативное время начала раб.дня']=pd.to_datetime(df_connection_time['Нормативное время начала раб.дня'])
df_connection_time['Фактич. время начала раб.дня']=pd.to_datetime(df_connection_time['Фактич. время начала раб.дня'])
df_connection_time['filtime']=df_connection_time['Нормативное время начала раб.дня']-df_connection_time['Фактич. время начала раб.дня']
filtime=pd.DataFrame(df_connection_time.groupby('id')['filtime'].mean().dt.total_seconds()).reset_index()
df_train = pd.merge(df_train, filtime , on="id", how='left')
df_train

,id,type,Вид образования,Число опозданий,CallTime,NumberOfCalls_worck,NumberOfCalls_free,NumberOfCallsfreesum,NumberOfCalls_x,NumberOfCalls_y,Время опоздания,filtime
0,ОРГ1-02050,2,4,5.0,4.810983e+18,546.0,546.0,NaN,176.0,370.0,2.361381e+17,276.332096
1,ОРГ1-02783,2,0,7.0,1.350599e+19,1902.0,1902.0,NaN,657.0,1245.0,2.532397e+17,-695.247628
2,ОРГ2-06173,0,0,22.0,NaN,NaN,NaN,NaN,NaN,NaN,1.200253e+18,-608.828839
3,ОРГ2-05359,2,10,6.0,NaN,NaN,NaN,NaN,NaN,NaN,1.657396e+17,2186.244912
4,ОРГ1-00803,2,0,35.0,1.388889e+17,7.0,7.0,NaN,1.0,6.0,9.288578e+17,-14381.555892
...,...,...,...,...,...,...,...,...,...,...,...,...
545,ОРГ1-02587,2,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
546,ОРГ1-02383,2,0,54.0,2.259805e+19,4570.0,4234.0,336.0,1692.0,2878.0,2.432262e+18,-7348.409864
547,ОРГ1-02862,2,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
548,ОРГ1-00471,2,-1,12.0,1.749719e+19,3917.0,3638.0,279.0,1088.0,2829.0,5.449973e+17,-58.303964


In [26]:
# Сброс ограничений на количество выводимых рядов
#pd.set_option('display.max_rows', None)
 
# Сброс ограничений на число столбцов
#pd.set_option('display.max_columns', None)
 
# Сброс ограничений на количество символов в записи
#pd.set_option('display.max_colwidth', None)

## Добавим Профессию

In [27]:
prof=pd.DataFrame(df_ed['Специальность'].value_counts()).reset_index()
profhed=prof.head(457)
profhed['profcod']=profhed.index
proftail=prof.tail(623)
proftail['profcod']=999
profhed=profhed.append(proftail)
profhed= profhed.rename(columns={'index': 'specialisation'})
uniq = profhed['specialisation'].values
fifa = profhed['profcod'].values
uniq_and_fifa = dict(zip(uniq, fifa))
uniq_and_fifa
df_ed['Специальность']=df_ed['Специальность'].map(uniq_and_fifa)
professionstart=df_ed[['id','Специальность']].groupby('id')['Специальность'].min()
professionend=df_ed[['id','Специальность']].groupby('id')['Специальность'].max()
df_train = pd.merge(df_train, professionstart , on="id", how='left')
df_train = pd.merge(df_train, professionend , on="id", how='left')
df_train

/tmp/ipykernel_33784/2507229956.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  profhed['profcod']=profhed.index
/tmp/ipykernel_33784/2507229956.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  proftail['profcod']=999
/tmp/ipykernel_33784/2507229956.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  profhed=profhed.append(proftail)


,id,type,Вид образования,Число опозданий,CallTime,NumberOfCalls_worck,NumberOfCalls_free,NumberOfCallsfreesum,NumberOfCalls_x,NumberOfCalls_y,Время опоздания,filtime,Специальность_x,Специальность_y
0,ОРГ1-02050,2,4,5.0,4.810983e+18,546.0,546.0,NaN,176.0,370.0,2.361381e+17,276.332096,21.0,21.0
1,ОРГ1-02783,2,0,7.0,1.350599e+19,1902.0,1902.0,NaN,657.0,1245.0,2.532397e+17,-695.247628,16.0,16.0
2,ОРГ2-06173,0,0,22.0,NaN,NaN,NaN,NaN,NaN,NaN,1.200253e+18,-608.828839,116.0,116.0
3,ОРГ2-05359,2,10,6.0,NaN,NaN,NaN,NaN,NaN,NaN,1.657396e+17,2186.244912,999.0,999.0
4,ОРГ1-00803,2,0,35.0,1.388889e+17,7.0,7.0,NaN,1.0,6.0,9.288578e+17,-14381.555892,365.0,365.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545,ОРГ1-02587,2,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
546,ОРГ1-02383,2,0,54.0,2.259805e+19,4570.0,4234.0,336.0,1692.0,2878.0,2.432262e+18,-7348.409864,24.0,24.0
547,ОРГ1-02862,2,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
548,ОРГ1-00471,2,-1,12.0,1.749719e+19,3917.0,3638.0,279.0,1088.0,2829.0,5.449973e+17,-58.303964,136.0,160.0


## Добавим rколичество  руководителей

In [28]:
df_ed

,Табельный номер руководителя,id,Вид образования,Специальность
0,NaN,ОРГ1-00131,Высшее образование - бакалавриат,6.0
1,NaN,ОРГ1-03220,Среднее профессиональное образование,135.0
2,NaN,ОРГ1-03008,Высшее образование,6.0
3,NaN,ОРГ1-00922,Высшее образование - бакалавриат,999.0
4,NaN,ОРГ1-00922,Высшее образование,35.0
...,...,...,...,...
5604,ОРГ2-09100,ОРГ2-08601,Начальное профессиональное образование,55.0
5605,ОРГ2-09100,ОРГ2-08622,Начальное профессиональное образование,NaN
5606,ОРГ2-09100,ОРГ2-04773,Среднее профессиональное образование,275.0
5607,ОРГ2-09100,ОРГ2-05048,"Высшее образование - специалитет, магистратура",257.0


In [29]:
boss=df_ed.groupby('id')['Табельный номер руководителя'].count()

In [30]:
df_train = pd.merge(df_train, boss , on="id", how='left')

# Добавим колличество завершенных заданий

In [31]:
df_tasks['Вид документа'].value_counts()

Служебная записка                              219740
Входящий                                        85972
Исходящий                                       76326
Приказ                                          43382
Внутренний документ                             27526
Договор                                         21082
Дополнительное соглашение                       20128
Акт                                             17999
Распоряжение                                     5898
Файл                                             5704
Протокол                                         3519
Исходящий РМЗ                                    2748
Внутренний документ для БП синхронизация 1С      2201
Запрос на внесение изменений                      665
Отчет по ВА                                       253
Name: Вид документа, dtype: int64

In [32]:
df_tasks.head(50)

,Статус по просрочке,Срок плановый,"Просрочено, дней",ДлительностьПросрочки,ID задачи,Вид документа,Дата старта задания,Дата завершения задания плановая,Дата завершения задания фактическая,Состояние задания,id
0,Без нарушения срока,NaN,0,без нарушения срока,E1DE844D-EE2D-4C41-AEDF-93F246749F0E,Служебная записка,"2021-12-10 00:00:00,000",NaN,"2021-12-10 00:00:00,000",Завершено,ОРГ1-02588
1,Без нарушения срока,NaN,0,без нарушения срока,7A92343C-8C9A-46E7-AC81-8F50F95009D0,Служебная записка,"2021-12-10 00:00:00,000",NaN,"2021-12-10 00:00:00,000",Завершено,ОРГ1-02588
2,Без нарушения срока,NaN,0,без нарушения срока,5CE64E52-D2D1-4DCC-B2C8-34734AA39AC0,Служебная записка,"2021-12-10 00:00:00,000",NaN,"2021-12-10 00:00:00,000",Завершено,ОРГ1-02588
3,Без нарушения срока,NaN,0,без нарушения срока,7A28F3DD-983F-4127-AB7F-6EDB85A69F1C,Служебная записка,"2021-12-10 00:00:00,000",NaN,"2021-12-10 00:00:00,000",Завершено,ОРГ1-02588
4,Без нарушения срока,NaN,0,без нарушения срока,ABFFAF61-12D8-44C2-B1F6-8402D174889E,Служебная записка,"2021-12-10 00:00:00,000",NaN,"2021-12-10 00:00:00,000",Завершено,ОРГ1-02588
5,Без нарушения срока,NaN,0,без нарушения срока,F229CA53-B362-42D7-8B35-F969EB14F421,Служебная записка,"2021-12-10 00:00:00,000",NaN,"2021-12-10 00:00:00,000",Завершено,ОРГ1-02588
6,Без нарушения срока,NaN,0,без нарушения срока,BE9F1F9E-B665-4306-AAFA-38980940C6BF,Служебная записка,"2021-12-10 00:00:00,000",NaN,"2021-12-10 00:00:00,000",Завершено,ОРГ1-02588
7,Без нарушения срока,NaN,0,без нарушения срока,D805905B-7176-44DA-9BE4-413C87396AE6,Служебная записка,"2021-12-10 00:00:00,000",NaN,"2021-12-10 00:00:00,000",Завершено,ОРГ1-02588
8,Без нарушения срока,NaN,0,без нарушения срока,AF0D0B62-EA68-489A-8710-3025BECBE93B,Служебная записка,"2021-12-10 00:00:00,000",NaN,"2021-12-10 00:00:00,000",Завершено,ОРГ1-02588
9,Без нарушения срока,NaN,0,без нарушения срока,5B267180-9C99-4326-9A2C-F0E5ECC12C7C,Служебная записка,"2021-12-10 00:00:00,000",NaN,"2021-12-10 00:00:00,000",Завершено,ОРГ1-02588


In [33]:
df_tasks['Статус по просрочке'].value_counts()

Без нарушения срока    454148
С нарушением срока      82512
Name: Статус по просрочке, dtype: int64

In [34]:
df_tasks = df_tasks.rename(columns={'Состояние задания': 'status_mission_count','Статус по просрочке':'статус_просрочек'})
status_mission_count=pd.DataFrame(df_tasks.query("status_mission_count=='Завершено'").groupby('id')['status_mission_count'].count()).reset_index().rename(columns={'status_mission_count': 'завершено'})
df_train = pd.merge(df_train,status_mission_count, on="id", how='left')

# Добавим колличество заданий в работе.

In [35]:
status_mission_count_in=pd.DataFrame(df_tasks.query("status_mission_count=='В работе'").groupby('id')['status_mission_count'].count()).reset_index().rename(columns={'status_mission_count': 'вработе'})
df_train = pd.merge(df_train,status_mission_count_in, on="id", how='left')
df_train

,id,type,Вид образования,Число опозданий,CallTime,NumberOfCalls_worck,NumberOfCalls_free,NumberOfCallsfreesum,NumberOfCalls_x,NumberOfCalls_y,Время опоздания,filtime,Специальность_x,Специальность_y,Табельный номер руководителя,завершено,вработе
0,ОРГ1-02050,2,4,5.0,4.810983e+18,546.0,546.0,NaN,176.0,370.0,2.361381e+17,276.332096,21.0,21.0,0.0,1.0,NaN
1,ОРГ1-02783,2,0,7.0,1.350599e+19,1902.0,1902.0,NaN,657.0,1245.0,2.532397e+17,-695.247628,16.0,16.0,1.0,306.0,NaN
2,ОРГ2-06173,0,0,22.0,NaN,NaN,NaN,NaN,NaN,NaN,1.200253e+18,-608.828839,116.0,116.0,1.0,NaN,NaN
3,ОРГ2-05359,2,10,6.0,NaN,NaN,NaN,NaN,NaN,NaN,1.657396e+17,2186.244912,999.0,999.0,1.0,NaN,NaN
4,ОРГ1-00803,2,0,35.0,1.388889e+17,7.0,7.0,NaN,1.0,6.0,9.288578e+17,-14381.555892,365.0,365.0,0.0,155.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545,ОРГ1-02587,2,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,123.0,NaN
546,ОРГ1-02383,2,0,54.0,2.259805e+19,4570.0,4234.0,336.0,1692.0,2878.0,2.432262e+18,-7348.409864,24.0,24.0,0.0,102.0,3.0
547,ОРГ1-02862,2,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
548,ОРГ1-00471,2,-1,12.0,1.749719e+19,3917.0,3638.0,279.0,1088.0,2829.0,5.449973e+17,-58.303964,136.0,160.0,0.0,33.0,NaN


# Добавим колличество заданий Не начато

In [36]:
status_mission_count_nn=pd.DataFrame(df_tasks.query("status_mission_count=='Не начато'").groupby('id')['status_mission_count'].count()).reset_index().rename(columns={'status_mission_count': 'неначато'})
df_train = pd.merge(df_train,status_mission_count_nn, on="id", how='left')
df_train

,id,type,Вид образования,Число опозданий,CallTime,NumberOfCalls_worck,NumberOfCalls_free,NumberOfCallsfreesum,NumberOfCalls_x,NumberOfCalls_y,Время опоздания,filtime,Специальность_x,Специальность_y,Табельный номер руководителя,завершено,вработе,неначато
0,ОРГ1-02050,2,4,5.0,4.810983e+18,546.0,546.0,NaN,176.0,370.0,2.361381e+17,276.332096,21.0,21.0,0.0,1.0,NaN,NaN
1,ОРГ1-02783,2,0,7.0,1.350599e+19,1902.0,1902.0,NaN,657.0,1245.0,2.532397e+17,-695.247628,16.0,16.0,1.0,306.0,NaN,NaN
2,ОРГ2-06173,0,0,22.0,NaN,NaN,NaN,NaN,NaN,NaN,1.200253e+18,-608.828839,116.0,116.0,1.0,NaN,NaN,NaN
3,ОРГ2-05359,2,10,6.0,NaN,NaN,NaN,NaN,NaN,NaN,1.657396e+17,2186.244912,999.0,999.0,1.0,NaN,NaN,NaN
4,ОРГ1-00803,2,0,35.0,1.388889e+17,7.0,7.0,NaN,1.0,6.0,9.288578e+17,-14381.555892,365.0,365.0,0.0,155.0,NaN,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545,ОРГ1-02587,2,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,123.0,NaN,16.0
546,ОРГ1-02383,2,0,54.0,2.259805e+19,4570.0,4234.0,336.0,1692.0,2878.0,2.432262e+18,-7348.409864,24.0,24.0,0.0,102.0,3.0,NaN
547,ОРГ1-02862,2,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
548,ОРГ1-00471,2,-1,12.0,1.749719e+19,3917.0,3638.0,279.0,1088.0,2829.0,5.449973e+17,-58.303964,136.0,160.0,0.0,33.0,NaN,NaN


# Добавим колличество заданий делегировано

In [37]:
status_mission_count_d=pd.DataFrame(df_tasks.query("status_mission_count=='Делегировано'").groupby('id')['status_mission_count'].count()).reset_index().rename(columns={'status_mission_count': 'делегировано'})
df_train = pd.merge(df_train,status_mission_count_d, on="id", how='left')
df_train

,id,type,Вид образования,Число опозданий,CallTime,NumberOfCalls_worck,NumberOfCalls_free,NumberOfCallsfreesum,NumberOfCalls_x,NumberOfCalls_y,Время опоздания,filtime,Специальность_x,Специальность_y,Табельный номер руководителя,завершено,вработе,неначато,делегировано
0,ОРГ1-02050,2,4,5.0,4.810983e+18,546.0,546.0,NaN,176.0,370.0,2.361381e+17,276.332096,21.0,21.0,0.0,1.0,NaN,NaN,NaN
1,ОРГ1-02783,2,0,7.0,1.350599e+19,1902.0,1902.0,NaN,657.0,1245.0,2.532397e+17,-695.247628,16.0,16.0,1.0,306.0,NaN,NaN,NaN
2,ОРГ2-06173,0,0,22.0,NaN,NaN,NaN,NaN,NaN,NaN,1.200253e+18,-608.828839,116.0,116.0,1.0,NaN,NaN,NaN,NaN
3,ОРГ2-05359,2,10,6.0,NaN,NaN,NaN,NaN,NaN,NaN,1.657396e+17,2186.244912,999.0,999.0,1.0,NaN,NaN,NaN,NaN
4,ОРГ1-00803,2,0,35.0,1.388889e+17,7.0,7.0,NaN,1.0,6.0,9.288578e+17,-14381.555892,365.0,365.0,0.0,155.0,NaN,4.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545,ОРГ1-02587,2,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,123.0,NaN,16.0,NaN
546,ОРГ1-02383,2,0,54.0,2.259805e+19,4570.0,4234.0,336.0,1692.0,2878.0,2.432262e+18,-7348.409864,24.0,24.0,0.0,102.0,3.0,NaN,NaN
547,ОРГ1-02862,2,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
548,ОРГ1-00471,2,-1,12.0,1.749719e+19,3917.0,3638.0,279.0,1088.0,2829.0,5.449973e+17,-58.303964,136.0,160.0,0.0,33.0,NaN,NaN,NaN


# Добавим колличество заданий На приёмке

In [38]:
status_mission_count_p=pd.DataFrame(df_tasks.query("status_mission_count=='На приёмке'").groupby('id')['status_mission_count'].count()).reset_index().rename(columns={'status_mission_count': 'наприемке'})
df_train = pd.merge(df_train,status_mission_count_p, on="id", how='left')
df_train


,id,type,Вид образования,Число опозданий,CallTime,NumberOfCalls_worck,NumberOfCalls_free,NumberOfCallsfreesum,NumberOfCalls_x,NumberOfCalls_y,Время опоздания,filtime,Специальность_x,Специальность_y,Табельный номер руководителя,завершено,вработе,неначато,делегировано,наприемке
0,ОРГ1-02050,2,4,5.0,4.810983e+18,546.0,546.0,NaN,176.0,370.0,2.361381e+17,276.332096,21.0,21.0,0.0,1.0,NaN,NaN,NaN,NaN
1,ОРГ1-02783,2,0,7.0,1.350599e+19,1902.0,1902.0,NaN,657.0,1245.0,2.532397e+17,-695.247628,16.0,16.0,1.0,306.0,NaN,NaN,NaN,NaN
2,ОРГ2-06173,0,0,22.0,NaN,NaN,NaN,NaN,NaN,NaN,1.200253e+18,-608.828839,116.0,116.0,1.0,NaN,NaN,NaN,NaN,NaN
3,ОРГ2-05359,2,10,6.0,NaN,NaN,NaN,NaN,NaN,NaN,1.657396e+17,2186.244912,999.0,999.0,1.0,NaN,NaN,NaN,NaN,NaN
4,ОРГ1-00803,2,0,35.0,1.388889e+17,7.0,7.0,NaN,1.0,6.0,9.288578e+17,-14381.555892,365.0,365.0,0.0,155.0,NaN,4.0,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545,ОРГ1-02587,2,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,123.0,NaN,16.0,NaN,NaN
546,ОРГ1-02383,2,0,54.0,2.259805e+19,4570.0,4234.0,336.0,1692.0,2878.0,2.432262e+18,-7348.409864,24.0,24.0,0.0,102.0,3.0,NaN,NaN,NaN
547,ОРГ1-02862,2,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
548,ОРГ1-00471,2,-1,12.0,1.749719e+19,3917.0,3638.0,279.0,1088.0,2829.0,5.449973e+17,-58.303964,136.0,160.0,0.0,33.0,NaN,NaN,NaN,NaN


# Добавим колличество заданий Отложено

In [39]:
status_mission_count_o=pd.DataFrame(df_tasks.query("status_mission_count=='Отложено'").groupby('id')['status_mission_count'].count()).reset_index().rename(columns={'status_mission_count': 'Отложено'})
df_train = pd.merge(df_train,status_mission_count_o, on="id", how='left')
df_train

,id,type,Вид образования,Число опозданий,CallTime,NumberOfCalls_worck,NumberOfCalls_free,NumberOfCallsfreesum,NumberOfCalls_x,NumberOfCalls_y,...,filtime,Специальность_x,Специальность_y,Табельный номер руководителя,завершено,вработе,неначато,делегировано,наприемке,Отложено
0,ОРГ1-02050,2,4,5.0,4.810983e+18,546.0,546.0,NaN,176.0,370.0,...,276.332096,21.0,21.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN
1,ОРГ1-02783,2,0,7.0,1.350599e+19,1902.0,1902.0,NaN,657.0,1245.0,...,-695.247628,16.0,16.0,1.0,306.0,NaN,NaN,NaN,NaN,NaN
2,ОРГ2-06173,0,0,22.0,NaN,NaN,NaN,NaN,NaN,NaN,...,-608.828839,116.0,116.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
3,ОРГ2-05359,2,10,6.0,NaN,NaN,NaN,NaN,NaN,NaN,...,2186.244912,999.0,999.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
4,ОРГ1-00803,2,0,35.0,1.388889e+17,7.0,7.0,NaN,1.0,6.0,...,-14381.555892,365.0,365.0,0.0,155.0,NaN,4.0,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545,ОРГ1-02587,2,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,123.0,NaN,16.0,NaN,NaN,NaN
546,ОРГ1-02383,2,0,54.0,2.259805e+19,4570.0,4234.0,336.0,1692.0,2878.0,...,-7348.409864,24.0,24.0,0.0,102.0,3.0,NaN,NaN,NaN,NaN
547,ОРГ1-02862,2,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
548,ОРГ1-00471,2,-1,12.0,1.749719e+19,3917.0,3638.0,279.0,1088.0,2829.0,...,-58.303964,136.0,160.0,0.0,33.0,NaN,NaN,NaN,NaN,NaN


## Добавим сумму просрочки дней!

In [40]:
day_ful_mission=pd.DataFrame(df_tasks.groupby('id')['Просрочено, дней'].sum()).reset_index().rename(columns={'Просрочено, дней': 'просрочено всего'})
df_train = pd.merge(df_train,day_ful_mission, on="id", how='left')
df_train

,id,type,Вид образования,Число опозданий,CallTime,NumberOfCalls_worck,NumberOfCalls_free,NumberOfCallsfreesum,NumberOfCalls_x,NumberOfCalls_y,...,Специальность_x,Специальность_y,Табельный номер руководителя,завершено,вработе,неначато,делегировано,наприемке,Отложено,просрочено всего
0,ОРГ1-02050,2,4,5.0,4.810983e+18,546.0,546.0,NaN,176.0,370.0,...,21.0,21.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,0.0
1,ОРГ1-02783,2,0,7.0,1.350599e+19,1902.0,1902.0,NaN,657.0,1245.0,...,16.0,16.0,1.0,306.0,NaN,NaN,NaN,NaN,NaN,861.0
2,ОРГ2-06173,0,0,22.0,NaN,NaN,NaN,NaN,NaN,NaN,...,116.0,116.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ОРГ2-05359,2,10,6.0,NaN,NaN,NaN,NaN,NaN,NaN,...,999.0,999.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ОРГ1-00803,2,0,35.0,1.388889e+17,7.0,7.0,NaN,1.0,6.0,...,365.0,365.0,0.0,155.0,NaN,4.0,1.0,NaN,NaN,1976.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545,ОРГ1-02587,2,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,123.0,NaN,16.0,NaN,NaN,NaN,3656.0
546,ОРГ1-02383,2,0,54.0,2.259805e+19,4570.0,4234.0,336.0,1692.0,2878.0,...,24.0,24.0,0.0,102.0,3.0,NaN,NaN,NaN,NaN,12199.0
547,ОРГ1-02862,2,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
548,ОРГ1-00471,2,-1,12.0,1.749719e+19,3917.0,3638.0,279.0,1088.0,2829.0,...,136.0,160.0,0.0,33.0,NaN,NaN,NaN,NaN,NaN,53.0


# Добавим статус по просрочке с нарушением без науршения и их отношение.

In [41]:
fail_count_b=pd.DataFrame(df_tasks.query('статус_просрочек=="Без нарушения срока"').groupby('id')['статус_просрочек'].count())
fail_count_p=pd.DataFrame(df_tasks.query('статус_просрочек=="С нарушением срока"').groupby('id')['статус_просрочек'].count()).rename(columns={'статус_просрочек': 'просрочено'})
df_train = pd.merge(df_train,fail_count_b, on="id", how='left')
df_train = pd.merge(df_train,fail_count_p, on="id", how='left')
df_train['bp']=df_train['статус_просрочек']/df_train['просрочено']

# Добавим общее время активности в сети

In [42]:
timennetwork_time=pd.DataFrame(timennetwork.groupby('id')['monitor_Time'].sum())
df_train = pd.merge(df_train,timennetwork_time, on="id", how='left')

# Добавим среднее время простоя монитора  

In [43]:
workingday['monitorTime_mean']=workingday['monitorTime']-workingday['activeTime']
monitor_stop=pd.DataFrame(workingday.groupby('id')['monitorTime_mean'].mean())
df_train = pd.merge(df_train,monitor_stop, on="id", how='left')

# Добавим счетчик начал рабочего дня

In [44]:
df_connection_time=df_connection_time.rename(columns={'Нормативное время начала раб.дня': 'начало_дня'})
start_day=pd.DataFrame(df_connection_time.groupby('id')['начало_дня'].count())
df_train = pd.merge(df_train,start_day, on="id", how='left')

In [45]:
df_train


,id,type,Вид образования,Число опозданий,CallTime,NumberOfCalls_worck,NumberOfCalls_free,NumberOfCallsfreesum,NumberOfCalls_x,NumberOfCalls_y,...,делегировано,наприемке,Отложено,просрочено всего,статус_просрочек,просрочено,bp,monitor_Time,monitorTime_mean,начало_дня
0,ОРГ1-02050,2,4,5.0,4.810983e+18,546.0,546.0,NaN,176.0,370.0,...,NaN,NaN,NaN,0.0,1.0,NaN,NaN,2762376.0,8073.693333,114.0
1,ОРГ1-02783,2,0,7.0,1.350599e+19,1902.0,1902.0,NaN,657.0,1245.0,...,NaN,NaN,NaN,861.0,256.0,50.0,5.120000,3667950.0,5404.023349,129.0
2,ОРГ2-06173,0,0,22.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,168.0
3,ОРГ2-05359,2,10,6.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2141948.0,596.958298,125.0
4,ОРГ1-00803,2,0,35.0,1.388889e+17,7.0,7.0,NaN,1.0,6.0,...,1.0,NaN,NaN,1976.0,141.0,19.0,7.421053,80139.0,35.529101,37.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545,ОРГ1-02587,2,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3656.0,93.0,46.0,2.021739,2369664.0,720.741639,NaN
546,ОРГ1-02383,2,0,54.0,2.259805e+19,4570.0,4234.0,336.0,1692.0,2878.0,...,NaN,NaN,NaN,12199.0,20.0,85.0,0.235294,1936264.0,6290.453890,162.0
547,ОРГ1-02862,2,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1642839.0,1102.914224,NaN
548,ОРГ1-00471,2,-1,12.0,1.749719e+19,3917.0,3638.0,279.0,1088.0,2829.0,...,NaN,NaN,NaN,53.0,25.0,8.0,3.125000,971065.0,15069.358263,137.0


# Добавим колличество уникальных профессий

In [46]:

prof_countis=pd.DataFrame(df_ed.groupby('id')['Специальность'].nunique())
df_train = pd.merge(df_train,prof_countis, on="id", how='left')
df_train

,id,type,Вид образования,Число опозданий,CallTime,NumberOfCalls_worck,NumberOfCalls_free,NumberOfCallsfreesum,NumberOfCalls_x,NumberOfCalls_y,...,наприемке,Отложено,просрочено всего,статус_просрочек,просрочено,bp,monitor_Time,monitorTime_mean,начало_дня,Специальность
0,ОРГ1-02050,2,4,5.0,4.810983e+18,546.0,546.0,NaN,176.0,370.0,...,NaN,NaN,0.0,1.0,NaN,NaN,2762376.0,8073.693333,114.0,1.0
1,ОРГ1-02783,2,0,7.0,1.350599e+19,1902.0,1902.0,NaN,657.0,1245.0,...,NaN,NaN,861.0,256.0,50.0,5.120000,3667950.0,5404.023349,129.0,1.0
2,ОРГ2-06173,0,0,22.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,168.0,1.0
3,ОРГ2-05359,2,10,6.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2141948.0,596.958298,125.0,1.0
4,ОРГ1-00803,2,0,35.0,1.388889e+17,7.0,7.0,NaN,1.0,6.0,...,NaN,NaN,1976.0,141.0,19.0,7.421053,80139.0,35.529101,37.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545,ОРГ1-02587,2,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,3656.0,93.0,46.0,2.021739,2369664.0,720.741639,NaN,NaN
546,ОРГ1-02383,2,0,54.0,2.259805e+19,4570.0,4234.0,336.0,1692.0,2878.0,...,NaN,NaN,12199.0,20.0,85.0,0.235294,1936264.0,6290.453890,162.0,1.0
547,ОРГ1-02862,2,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1642839.0,1102.914224,NaN,NaN
548,ОРГ1-00471,2,-1,12.0,1.749719e+19,3917.0,3638.0,279.0,1088.0,2829.0,...,NaN,NaN,53.0,25.0,8.0,3.125000,971065.0,15069.358263,137.0,2.0


# Добавим оборотные виды документов.

In [47]:
df_tasks
documents=pd.DataFrame(df_tasks.groupby('id')['Вид документа'].nunique())
df_train = pd.merge(df_train,documents, on="id", how='left')
df_train

,id,type,Вид образования,Число опозданий,CallTime,NumberOfCalls_worck,NumberOfCalls_free,NumberOfCallsfreesum,NumberOfCalls_x,NumberOfCalls_y,...,Отложено,просрочено всего,статус_просрочек,просрочено,bp,monitor_Time,monitorTime_mean,начало_дня,Специальность,Вид документа
0,ОРГ1-02050,2,4,5.0,4.810983e+18,546.0,546.0,NaN,176.0,370.0,...,NaN,0.0,1.0,NaN,NaN,2762376.0,8073.693333,114.0,1.0,1.0
1,ОРГ1-02783,2,0,7.0,1.350599e+19,1902.0,1902.0,NaN,657.0,1245.0,...,NaN,861.0,256.0,50.0,5.120000,3667950.0,5404.023349,129.0,1.0,5.0
2,ОРГ2-06173,0,0,22.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,168.0,1.0,NaN
3,ОРГ2-05359,2,10,6.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2141948.0,596.958298,125.0,1.0,NaN
4,ОРГ1-00803,2,0,35.0,1.388889e+17,7.0,7.0,NaN,1.0,6.0,...,NaN,1976.0,141.0,19.0,7.421053,80139.0,35.529101,37.0,1.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545,ОРГ1-02587,2,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,3656.0,93.0,46.0,2.021739,2369664.0,720.741639,NaN,NaN,7.0
546,ОРГ1-02383,2,0,54.0,2.259805e+19,4570.0,4234.0,336.0,1692.0,2878.0,...,NaN,12199.0,20.0,85.0,0.235294,1936264.0,6290.453890,162.0,1.0,5.0
547,ОРГ1-02862,2,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1642839.0,1102.914224,NaN,NaN,NaN
548,ОРГ1-00471,2,-1,12.0,1.749719e+19,3917.0,3638.0,279.0,1088.0,2829.0,...,NaN,53.0,25.0,8.0,3.125000,971065.0,15069.358263,137.0,2.0,5.0


# Добавим месяцы звонков

In [48]:
df_calls['monday']=pd.to_datetime(df_calls["Date"]).dt.month
monday_calls=pd.DataFrame(df_calls.groupby('id')['monday'].nunique())
df_train = pd.merge(df_train,monday_calls, on="id", how='left')
df_train

,id,type,Вид образования,Число опозданий,CallTime,NumberOfCalls_worck,NumberOfCalls_free,NumberOfCallsfreesum,NumberOfCalls_x,NumberOfCalls_y,...,просрочено всего,статус_просрочек,просрочено,bp,monitor_Time,monitorTime_mean,начало_дня,Специальность,Вид документа,monday
0,ОРГ1-02050,2,4,5.0,4.810983e+18,546.0,546.0,NaN,176.0,370.0,...,0.0,1.0,NaN,NaN,2762376.0,8073.693333,114.0,1.0,1.0,10.0
1,ОРГ1-02783,2,0,7.0,1.350599e+19,1902.0,1902.0,NaN,657.0,1245.0,...,861.0,256.0,50.0,5.120000,3667950.0,5404.023349,129.0,1.0,5.0,10.0
2,ОРГ2-06173,0,0,22.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,168.0,1.0,NaN,NaN
3,ОРГ2-05359,2,10,6.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2141948.0,596.958298,125.0,1.0,NaN,NaN
4,ОРГ1-00803,2,0,35.0,1.388889e+17,7.0,7.0,NaN,1.0,6.0,...,1976.0,141.0,19.0,7.421053,80139.0,35.529101,37.0,1.0,5.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545,ОРГ1-02587,2,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3656.0,93.0,46.0,2.021739,2369664.0,720.741639,NaN,NaN,7.0,NaN
546,ОРГ1-02383,2,0,54.0,2.259805e+19,4570.0,4234.0,336.0,1692.0,2878.0,...,12199.0,20.0,85.0,0.235294,1936264.0,6290.453890,162.0,1.0,5.0,12.0
547,ОРГ1-02862,2,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1642839.0,1102.914224,NaN,NaN,NaN,NaN
548,ОРГ1-00471,2,-1,12.0,1.749719e+19,3917.0,3638.0,279.0,1088.0,2829.0,...,53.0,25.0,8.0,3.125000,971065.0,15069.358263,137.0,2.0,5.0,12.0


# Добавим месяцы в сети

In [49]:
timennetwork['mondayW']=pd.to_datetime(timennetwork['startTime']).dt.month
monday_worck=pd.DataFrame(timennetwork.groupby('id')['mondayW'].nunique())     
df_train = pd.merge(df_train,monday_worck, on="id", how='left')
df_train                                      

,id,type,Вид образования,Число опозданий,CallTime,NumberOfCalls_worck,NumberOfCalls_free,NumberOfCallsfreesum,NumberOfCalls_x,NumberOfCalls_y,...,статус_просрочек,просрочено,bp,monitor_Time,monitorTime_mean,начало_дня,Специальность,Вид документа,monday,mondayW
0,ОРГ1-02050,2,4,5.0,4.810983e+18,546.0,546.0,NaN,176.0,370.0,...,1.0,NaN,NaN,2762376.0,8073.693333,114.0,1.0,1.0,10.0,7.0
1,ОРГ1-02783,2,0,7.0,1.350599e+19,1902.0,1902.0,NaN,657.0,1245.0,...,256.0,50.0,5.120000,3667950.0,5404.023349,129.0,1.0,5.0,10.0,7.0
2,ОРГ2-06173,0,0,22.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,168.0,1.0,NaN,NaN,NaN
3,ОРГ2-05359,2,10,6.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2141948.0,596.958298,125.0,1.0,NaN,NaN,7.0
4,ОРГ1-00803,2,0,35.0,1.388889e+17,7.0,7.0,NaN,1.0,6.0,...,141.0,19.0,7.421053,80139.0,35.529101,37.0,1.0,5.0,3.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545,ОРГ1-02587,2,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,93.0,46.0,2.021739,2369664.0,720.741639,NaN,NaN,7.0,NaN,7.0
546,ОРГ1-02383,2,0,54.0,2.259805e+19,4570.0,4234.0,336.0,1692.0,2878.0,...,20.0,85.0,0.235294,1936264.0,6290.453890,162.0,1.0,5.0,12.0,7.0
547,ОРГ1-02862,2,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1642839.0,1102.914224,NaN,NaN,NaN,NaN,5.0
548,ОРГ1-00471,2,-1,12.0,1.749719e+19,3917.0,3638.0,279.0,1088.0,2829.0,...,25.0,8.0,3.125000,971065.0,15069.358263,137.0,2.0,5.0,12.0,7.0


# Добавим месяцы активности за монитором.

In [50]:
workingday['mondayWork']=pd.to_datetime(workingday['startTime']).dt.month
monday_Work=pd.DataFrame(workingday.groupby('id')['mondayWork'].nunique())     
df_train = pd.merge(df_train,monday_Work, on="id", how='left')
df_train              

,id,type,Вид образования,Число опозданий,CallTime,NumberOfCalls_worck,NumberOfCalls_free,NumberOfCallsfreesum,NumberOfCalls_x,NumberOfCalls_y,...,просрочено,bp,monitor_Time,monitorTime_mean,начало_дня,Специальность,Вид документа,monday,mondayW,mondayWork
0,ОРГ1-02050,2,4,5.0,4.810983e+18,546.0,546.0,NaN,176.0,370.0,...,NaN,NaN,2762376.0,8073.693333,114.0,1.0,1.0,10.0,7.0,7.0
1,ОРГ1-02783,2,0,7.0,1.350599e+19,1902.0,1902.0,NaN,657.0,1245.0,...,50.0,5.120000,3667950.0,5404.023349,129.0,1.0,5.0,10.0,7.0,7.0
2,ОРГ2-06173,0,0,22.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,168.0,1.0,NaN,NaN,NaN,NaN
3,ОРГ2-05359,2,10,6.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2141948.0,596.958298,125.0,1.0,NaN,NaN,7.0,7.0
4,ОРГ1-00803,2,0,35.0,1.388889e+17,7.0,7.0,NaN,1.0,6.0,...,19.0,7.421053,80139.0,35.529101,37.0,1.0,5.0,3.0,7.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545,ОРГ1-02587,2,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,46.0,2.021739,2369664.0,720.741639,NaN,NaN,7.0,NaN,7.0,7.0
546,ОРГ1-02383,2,0,54.0,2.259805e+19,4570.0,4234.0,336.0,1692.0,2878.0,...,85.0,0.235294,1936264.0,6290.453890,162.0,1.0,5.0,12.0,7.0,7.0
547,ОРГ1-02862,2,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1642839.0,1102.914224,NaN,NaN,NaN,NaN,5.0,5.0
548,ОРГ1-00471,2,-1,12.0,1.749719e+19,3917.0,3638.0,279.0,1088.0,2829.0,...,8.0,3.125000,971065.0,15069.358263,137.0,2.0,5.0,12.0,7.0,7.0


In [51]:
workingday

,startTime,activeTime,Вых/Будни,monitorTime,id,monitorTime_mean,mondayWork
0,"2021-11-30 00:00:00,000",2,Будни,2,ОРГ1-01553,0,11
1,"2021-11-30 00:00:00,000",2,Будни,2,ОРГ1-02112,0,11
2,"2021-11-30 00:00:00,000",2,Будни,2,ОРГ1-02112,0,11
3,"2021-11-30 00:00:00,000",2,Будни,2,ОРГ1-01846,0,11
4,"2021-11-30 00:00:00,000",2,Будни,2,ОРГ1-01846,0,11
...,...,...,...,...,...,...,...
1971922,"2021-12-26 00:00:00,000",6,Выходные дни,6,ОРГ1-03092,0,12
1971923,"2021-12-26 00:00:00,000",641,Выходные дни,1243,ОРГ1-03092,602,12
1971924,"2021-12-26 00:00:00,000",426,Выходные дни,426,ОРГ1-03092,0,12
1971925,"2021-12-26 00:00:00,000",10,Выходные дни,10,ОРГ1-03092,0,12


## Добавим  дельту дат звонка.

In [52]:
df_calls
one_cals=pd.DataFrame(df_calls.groupby('id')['Date'].min()).rename(columns={'Date': 'one_cals'})
last_cals=pd.DataFrame(df_calls.groupby('id')['Date'].max()).rename(columns={'Date': 'last_cals'})
last_cals['last_cals_delta']=((pd.to_datetime(last_cals['last_cals'])-pd.to_datetime(one_cals['one_cals'])).dt.days.astype('int'))
last_cals=last_cals.drop(['last_cals'], axis = 1)
df_train = pd.merge(df_train,last_cals, on="id", how='left')
df_train              

,id,type,Вид образования,Число опозданий,CallTime,NumberOfCalls_worck,NumberOfCalls_free,NumberOfCallsfreesum,NumberOfCalls_x,NumberOfCalls_y,...,bp,monitor_Time,monitorTime_mean,начало_дня,Специальность,Вид документа,monday,mondayW,mondayWork,last_cals_delta
0,ОРГ1-02050,2,4,5.0,4.810983e+18,546.0,546.0,NaN,176.0,370.0,...,NaN,2762376.0,8073.693333,114.0,1.0,1.0,10.0,7.0,7.0,352.0
1,ОРГ1-02783,2,0,7.0,1.350599e+19,1902.0,1902.0,NaN,657.0,1245.0,...,5.120000,3667950.0,5404.023349,129.0,1.0,5.0,10.0,7.0,7.0,336.0
2,ОРГ2-06173,0,0,22.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,168.0,1.0,NaN,NaN,NaN,NaN,NaN
3,ОРГ2-05359,2,10,6.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2141948.0,596.958298,125.0,1.0,NaN,NaN,7.0,7.0,NaN
4,ОРГ1-00803,2,0,35.0,1.388889e+17,7.0,7.0,NaN,1.0,6.0,...,7.421053,80139.0,35.529101,37.0,1.0,5.0,3.0,7.0,7.0,145.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545,ОРГ1-02587,2,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.021739,2369664.0,720.741639,NaN,NaN,7.0,NaN,7.0,7.0,NaN
546,ОРГ1-02383,2,0,54.0,2.259805e+19,4570.0,4234.0,336.0,1692.0,2878.0,...,0.235294,1936264.0,6290.453890,162.0,1.0,5.0,12.0,7.0,7.0,355.0
547,ОРГ1-02862,2,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1642839.0,1102.914224,NaN,NaN,NaN,NaN,5.0,5.0,NaN
548,ОРГ1-00471,2,-1,12.0,1.749719e+19,3917.0,3638.0,279.0,1088.0,2829.0,...,3.125000,971065.0,15069.358263,137.0,2.0,5.0,12.0,7.0,7.0,352.0


In [53]:
df_calls.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 407867 entries, 0 to 407866
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           407867 non-null  object 
 1   CallTime       407867 non-null  float64
 2   NumberOfCalls  407867 non-null  int64  
 3   statusworkfre  392655 non-null  object 
 4   InOut          407867 non-null  object 
 5   id             407867 non-null  object 
 6   monday         407867 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 21.8+ MB


# Добавим дельту сети

In [54]:
timennetwork

,Вых/Будни,monitor_Time,startTime,id,mondayW
0,Будни,300,"2021-08-16 00:00:00,000",ОРГ1-01402,8
1,Будни,300,"2021-08-18 00:00:00,000",ОРГ1-01402,8
2,Будни,300,"2021-08-19 00:00:00,000",ОРГ1-01402,8
3,Будни,300,"2021-08-23 00:00:00,000",ОРГ1-01402,8
4,Будни,300,"2021-08-26 00:00:00,000",ОРГ1-01402,8
...,...,...,...,...,...
216105,Выходные дни,300,"2021-11-05 00:00:00,000",ОРГ1-01770,11
216106,Выходные дни,300,"2021-12-26 00:00:00,000",ОРГ1-02799,12
216107,Выходные дни,300,"2021-11-06 00:00:00,000",ОРГ1-00933,11
216108,Выходные дни,300,"2021-11-28 00:00:00,000",ОРГ1-01630,11


In [55]:
df_calls
one_net=pd.DataFrame(timennetwork.groupby('id')['startTime'].min()).rename(columns={'startTime': 'one_net'})
last_net=pd.DataFrame(timennetwork.groupby('id')['startTime'].max()).rename(columns={'startTime': 'last_net'})
last_net['last_net_delta']=((pd.to_datetime(last_net['last_net'])-pd.to_datetime(one_net['one_net'])).dt.days.astype('int'))
last_net=last_net.drop(['last_net'], axis = 1)
df_train = pd.merge(df_train,last_net, on="id", how='left')
df_train              

,id,type,Вид образования,Число опозданий,CallTime,NumberOfCalls_worck,NumberOfCalls_free,NumberOfCallsfreesum,NumberOfCalls_x,NumberOfCalls_y,...,monitor_Time,monitorTime_mean,начало_дня,Специальность,Вид документа,monday,mondayW,mondayWork,last_cals_delta,last_net_delta
0,ОРГ1-02050,2,4,5.0,4.810983e+18,546.0,546.0,NaN,176.0,370.0,...,2762376.0,8073.693333,114.0,1.0,1.0,10.0,7.0,7.0,352.0,191.0
1,ОРГ1-02783,2,0,7.0,1.350599e+19,1902.0,1902.0,NaN,657.0,1245.0,...,3667950.0,5404.023349,129.0,1.0,5.0,10.0,7.0,7.0,336.0,191.0
2,ОРГ2-06173,0,0,22.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,168.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
3,ОРГ2-05359,2,10,6.0,NaN,NaN,NaN,NaN,NaN,NaN,...,2141948.0,596.958298,125.0,1.0,NaN,NaN,7.0,7.0,NaN,190.0
4,ОРГ1-00803,2,0,35.0,1.388889e+17,7.0,7.0,NaN,1.0,6.0,...,80139.0,35.529101,37.0,1.0,5.0,3.0,7.0,7.0,145.0,176.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545,ОРГ1-02587,2,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2369664.0,720.741639,NaN,NaN,7.0,NaN,7.0,7.0,NaN,165.0
546,ОРГ1-02383,2,0,54.0,2.259805e+19,4570.0,4234.0,336.0,1692.0,2878.0,...,1936264.0,6290.453890,162.0,1.0,5.0,12.0,7.0,7.0,355.0,192.0
547,ОРГ1-02862,2,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1642839.0,1102.914224,NaN,NaN,NaN,NaN,5.0,5.0,NaN,148.0
548,ОРГ1-00471,2,-1,12.0,1.749719e+19,3917.0,3638.0,279.0,1088.0,2829.0,...,971065.0,15069.358263,137.0,2.0,5.0,12.0,7.0,7.0,352.0,191.0


# Добавим дельту работы за компьютером

In [56]:
workingday

,startTime,activeTime,Вых/Будни,monitorTime,id,monitorTime_mean,mondayWork
0,"2021-11-30 00:00:00,000",2,Будни,2,ОРГ1-01553,0,11
1,"2021-11-30 00:00:00,000",2,Будни,2,ОРГ1-02112,0,11
2,"2021-11-30 00:00:00,000",2,Будни,2,ОРГ1-02112,0,11
3,"2021-11-30 00:00:00,000",2,Будни,2,ОРГ1-01846,0,11
4,"2021-11-30 00:00:00,000",2,Будни,2,ОРГ1-01846,0,11
...,...,...,...,...,...,...,...
1971922,"2021-12-26 00:00:00,000",6,Выходные дни,6,ОРГ1-03092,0,12
1971923,"2021-12-26 00:00:00,000",641,Выходные дни,1243,ОРГ1-03092,602,12
1971924,"2021-12-26 00:00:00,000",426,Выходные дни,426,ОРГ1-03092,0,12
1971925,"2021-12-26 00:00:00,000",10,Выходные дни,10,ОРГ1-03092,0,12


In [57]:

one_worck=pd.DataFrame(workingday.groupby('id')['startTime'].min()).rename(columns={'startTime': 'one_worck'})
last_worck=pd.DataFrame(workingday.groupby('id')['startTime'].max()).rename(columns={'startTime': 'last_worck'})
last_worck['last_worck']
one_worck['one_worck']
last_worck['last_worck_delta']=((pd.to_datetime(last_worck['last_worck'])-pd.to_datetime(one_worck['one_worck'])).dt.days.astype('int'))
last_worck=last_worck.drop(['last_worck'], axis = 1)
df_train = pd.merge(df_train,last_worck, on="id", how='left')
df_train              

,id,type,Вид образования,Число опозданий,CallTime,NumberOfCalls_worck,NumberOfCalls_free,NumberOfCallsfreesum,NumberOfCalls_x,NumberOfCalls_y,...,monitorTime_mean,начало_дня,Специальность,Вид документа,monday,mondayW,mondayWork,last_cals_delta,last_net_delta,last_worck_delta
0,ОРГ1-02050,2,4,5.0,4.810983e+18,546.0,546.0,NaN,176.0,370.0,...,8073.693333,114.0,1.0,1.0,10.0,7.0,7.0,352.0,191.0,192.0
1,ОРГ1-02783,2,0,7.0,1.350599e+19,1902.0,1902.0,NaN,657.0,1245.0,...,5404.023349,129.0,1.0,5.0,10.0,7.0,7.0,336.0,191.0,193.0
2,ОРГ2-06173,0,0,22.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,168.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ОРГ2-05359,2,10,6.0,NaN,NaN,NaN,NaN,NaN,NaN,...,596.958298,125.0,1.0,NaN,NaN,7.0,7.0,NaN,190.0,190.0
4,ОРГ1-00803,2,0,35.0,1.388889e+17,7.0,7.0,NaN,1.0,6.0,...,35.529101,37.0,1.0,5.0,3.0,7.0,7.0,145.0,176.0,183.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545,ОРГ1-02587,2,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,720.741639,NaN,NaN,7.0,NaN,7.0,7.0,NaN,165.0,165.0
546,ОРГ1-02383,2,0,54.0,2.259805e+19,4570.0,4234.0,336.0,1692.0,2878.0,...,6290.453890,162.0,1.0,5.0,12.0,7.0,7.0,355.0,192.0,193.0
547,ОРГ1-02862,2,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1102.914224,NaN,NaN,NaN,NaN,5.0,5.0,NaN,148.0,149.0
548,ОРГ1-00471,2,-1,12.0,1.749719e+19,3917.0,3638.0,279.0,1088.0,2829.0,...,15069.358263,137.0,2.0,5.0,12.0,7.0,7.0,352.0,191.0,193.0


# Звонки по месяцам

In [58]:
for i in range(1,13):
    calls_month_sum=pd.DataFrame(df_calls.query('monday==@i').groupby('id')['CallTime'].sum()).rename(columns={'CallTime': 'calls_month_sum'+ str(i)})
    df_train = pd.merge(df_train,calls_month_sum, on="id", how='left')
df_train

,id,type,Вид образования,Число опозданий,CallTime,NumberOfCalls_worck,NumberOfCalls_free,NumberOfCallsfreesum,NumberOfCalls_x,NumberOfCalls_y,...,calls_month_sum3,calls_month_sum4,calls_month_sum5,calls_month_sum6,calls_month_sum7,calls_month_sum8,calls_month_sum9,calls_month_sum10,calls_month_sum11,calls_month_sum12
0,ОРГ1-02050,2,4,5.0,4.810983e+18,546.0,546.0,NaN,176.0,370.0,...,1.519733e+18,1.007389e+18,NaN,1.206536e+17,NaN,9.694167e+16,2.763889e+17,4.033333e+16,4.166667e+16,5.102778e+16
1,ОРГ1-02783,2,0,7.0,1.350599e+19,1902.0,1902.0,NaN,657.0,1245.0,...,NaN,1.657500e+04,4.597222e+17,1.175039e+18,1.947472e+18,1.601148e+18,1.637858e+18,2.518600e+18,1.918786e+18,2.219869e+18
2,ОРГ2-06173,0,0,22.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ОРГ2-05359,2,10,6.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ОРГ1-00803,2,0,35.0,1.388889e+17,7.0,7.0,NaN,1.0,6.0,...,8.333333e+16,NaN,NaN,5.555556e+16,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545,ОРГ1-02587,2,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
546,ОРГ1-02383,2,0,54.0,2.259805e+19,4570.0,4234.0,336.0,1692.0,2878.0,...,1.931925e+18,1.834284e+18,3.007344e+18,1.347722e+18,1.635172e+18,1.417278e+18,2.345098e+18,1.959570e+18,2.173118e+18,2.098118e+18
547,ОРГ1-02862,2,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
548,ОРГ1-00471,2,-1,12.0,1.749719e+19,3917.0,3638.0,279.0,1088.0,2829.0,...,1.159619e+18,1.809381e+18,7.202083e+17,6.128667e+17,1.629948e+18,9.573861e+17,2.032422e+18,1.852275e+18,8.429722e+17,2.448305e+18


# Даты окончаний заданий по месмяцам.


In [59]:
df_tasks['month']=pd.to_datetime(df_tasks['Дата завершения задания фактическая']).dt.month
for i in range(1,13):
    tasks_month_sum=pd.DataFrame(df_tasks.query('month==@i').groupby('id')['Дата завершения задания фактическая'].count()).rename(columns={'Дата завершения задания фактическая': 'tasks_month_sum'+ str(i)})
    df_train = pd.merge(df_train,tasks_month_sum, on="id", how='left')
df_train

,id,type,Вид образования,Число опозданий,CallTime,NumberOfCalls_worck,NumberOfCalls_free,NumberOfCallsfreesum,NumberOfCalls_x,NumberOfCalls_y,...,tasks_month_sum3,tasks_month_sum4,tasks_month_sum5,tasks_month_sum6,tasks_month_sum7,tasks_month_sum8,tasks_month_sum9,tasks_month_sum10,tasks_month_sum11,tasks_month_sum12
0,ОРГ1-02050,2,4,5.0,4.810983e+18,546.0,546.0,NaN,176.0,370.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ОРГ1-02783,2,0,7.0,1.350599e+19,1902.0,1902.0,NaN,657.0,1245.0,...,NaN,NaN,2.0,10.0,54.0,53.0,95.0,31.0,17.0,34.0
2,ОРГ2-06173,0,0,22.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ОРГ2-05359,2,10,6.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ОРГ1-00803,2,0,35.0,1.388889e+17,7.0,7.0,NaN,1.0,6.0,...,9.0,14.0,4.0,7.0,17.0,21.0,15.0,23.0,13.0,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545,ОРГ1-02587,2,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,11.0,6.0,12.0,13.0,14.0,8.0,12.0,7.0,12.0,NaN
546,ОРГ1-02383,2,0,54.0,2.259805e+19,4570.0,4234.0,336.0,1692.0,2878.0,...,3.0,38.0,1.0,1.0,1.0,NaN,1.0,39.0,5.0,4.0
547,ОРГ1-02862,2,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
548,ОРГ1-00471,2,-1,12.0,1.749719e+19,3917.0,3638.0,279.0,1088.0,2829.0,...,3.0,1.0,NaN,2.0,NaN,1.0,NaN,NaN,1.0,16.0


# Добавим сеть по месяцам

In [60]:
for i in range(6,13):
    net_month_sum=pd.DataFrame(timennetwork.query('mondayW==@i').groupby('id')['monitor_Time'].sum()).rename(columns={'monitor_Time': 'net_month_sum'+ str(i)})
    df_train = pd.merge(df_train,net_month_sum, on="id", how='left')
df_train

,id,type,Вид образования,Число опозданий,CallTime,NumberOfCalls_worck,NumberOfCalls_free,NumberOfCallsfreesum,NumberOfCalls_x,NumberOfCalls_y,...,tasks_month_sum10,tasks_month_sum11,tasks_month_sum12,net_month_sum6,net_month_sum7,net_month_sum8,net_month_sum9,net_month_sum10,net_month_sum11,net_month_sum12
0,ОРГ1-02050,2,4,5.0,4.810983e+18,546.0,546.0,NaN,176.0,370.0,...,NaN,NaN,NaN,180697.0,249739.0,536816.0,519464.0,300484.0,511886.0,463290.0
1,ОРГ1-02783,2,0,7.0,1.350599e+19,1902.0,1902.0,NaN,657.0,1245.0,...,31.0,17.0,34.0,217747.0,625124.0,512029.0,671010.0,586915.0,564472.0,490653.0
2,ОРГ2-06173,0,0,22.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ОРГ2-05359,2,10,6.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,209210.0,515112.0,900.0,240658.0,481386.0,409011.0,285671.0
4,ОРГ1-00803,2,0,35.0,1.388889e+17,7.0,7.0,NaN,1.0,6.0,...,23.0,13.0,21.0,6309.0,28810.0,14106.0,1803.0,11707.0,10804.0,6600.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545,ОРГ1-02587,2,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.0,12.0,NaN,160529.0,465496.0,317776.0,239471.0,594886.0,591204.0,302.0
546,ОРГ1-02383,2,0,54.0,2.259805e+19,4570.0,4234.0,336.0,1692.0,2878.0,...,39.0,5.0,4.0,224512.0,402530.0,64231.0,381423.0,467431.0,249400.0,146737.0
547,ОРГ1-02862,2,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,355570.0,344778.0,352567.0,301268.0,288656.0
548,ОРГ1-00471,2,-1,12.0,1.749719e+19,3917.0,3638.0,279.0,1088.0,2829.0,...,NaN,1.0,16.0,84326.0,188753.0,108330.0,186349.0,147035.0,75028.0,181244.0


# Добавим работу за компьютером по месяцам

In [61]:
for i in range(6,13):
    worck_month_sum=pd.DataFrame(workingday.query('mondayWork==@i').groupby('id')['monitorTime'].sum()).rename(columns={'monitorTime': 'worck_month_sum'+ str(i)})
    df_train = pd.merge(df_train,worck_month_sum, on="id", how='left')
df_train

,id,type,Вид образования,Число опозданий,CallTime,NumberOfCalls_worck,NumberOfCalls_free,NumberOfCallsfreesum,NumberOfCalls_x,NumberOfCalls_y,...,net_month_sum10,net_month_sum11,net_month_sum12,worck_month_sum6,worck_month_sum7,worck_month_sum8,worck_month_sum9,worck_month_sum10,worck_month_sum11,worck_month_sum12
0,ОРГ1-02050,2,4,5.0,4.810983e+18,546.0,546.0,NaN,176.0,370.0,...,300484.0,511886.0,463290.0,482661.0,2371663.0,2215419.0,1384940.0,2233751.0,1856282.0,1699473.0
1,ОРГ1-02783,2,0,7.0,1.350599e+19,1902.0,1902.0,NaN,657.0,1245.0,...,586915.0,564472.0,490653.0,250008.0,1117199.0,819488.0,3006491.0,2581386.0,1931613.0,1752540.0
2,ОРГ2-06173,0,0,22.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ОРГ2-05359,2,10,6.0,NaN,NaN,NaN,NaN,NaN,NaN,...,481386.0,409011.0,285671.0,246629.0,579031.0,350.0,264201.0,560542.0,413320.0,280000.0
4,ОРГ1-00803,2,0,35.0,1.388889e+17,7.0,7.0,NaN,1.0,6.0,...,11707.0,10804.0,6600.0,3620.0,15088.0,5054.0,1599.0,5713.0,4669.0,3137.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545,ОРГ1-02587,2,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,594886.0,591204.0,302.0,188573.0,584673.0,360356.0,300196.0,741380.0,719085.0,93.0
546,ОРГ1-02383,2,0,54.0,2.259805e+19,4570.0,4234.0,336.0,1692.0,2878.0,...,467431.0,249400.0,146737.0,883358.0,2266861.0,583678.0,1751236.0,2319656.0,1698734.0,956273.0
547,ОРГ1-02862,2,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,352567.0,301268.0,288656.0,NaN,NaN,419767.0,468000.0,502917.0,409783.0,415297.0
548,ОРГ1-00471,2,-1,12.0,1.749719e+19,3917.0,3638.0,279.0,1088.0,2829.0,...,147035.0,75028.0,181244.0,793119.0,2279778.0,1819426.0,2434422.0,2644993.0,922389.0,2635880.0


## Заменим пропуски на нули

In [62]:
df_train = df_train.fillna(0)

In [63]:
df_train.append(df_train)
#df_train.append(df_train)
#df_train.append(df_train)


/tmp/ipykernel_33784/1624600992.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_train.append(df_train)


,id,type,Вид образования,Число опозданий,CallTime,NumberOfCalls_worck,NumberOfCalls_free,NumberOfCallsfreesum,NumberOfCalls_x,NumberOfCalls_y,...,net_month_sum10,net_month_sum11,net_month_sum12,worck_month_sum6,worck_month_sum7,worck_month_sum8,worck_month_sum9,worck_month_sum10,worck_month_sum11,worck_month_sum12
0,ОРГ1-02050,2,4,5.0,4.810983e+18,546.0,546.0,0.0,176.0,370.0,...,300484.0,511886.0,463290.0,482661.0,2371663.0,2215419.0,1384940.0,2233751.0,1856282.0,1699473.0
1,ОРГ1-02783,2,0,7.0,1.350599e+19,1902.0,1902.0,0.0,657.0,1245.0,...,586915.0,564472.0,490653.0,250008.0,1117199.0,819488.0,3006491.0,2581386.0,1931613.0,1752540.0
2,ОРГ2-06173,0,0,22.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ОРГ2-05359,2,10,6.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,...,481386.0,409011.0,285671.0,246629.0,579031.0,350.0,264201.0,560542.0,413320.0,280000.0
4,ОРГ1-00803,2,0,35.0,1.388889e+17,7.0,7.0,0.0,1.0,6.0,...,11707.0,10804.0,6600.0,3620.0,15088.0,5054.0,1599.0,5713.0,4669.0,3137.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545,ОРГ1-02587,2,-1,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,...,594886.0,591204.0,302.0,188573.0,584673.0,360356.0,300196.0,741380.0,719085.0,93.0
546,ОРГ1-02383,2,0,54.0,2.259805e+19,4570.0,4234.0,336.0,1692.0,2878.0,...,467431.0,249400.0,146737.0,883358.0,2266861.0,583678.0,1751236.0,2319656.0,1698734.0,956273.0
547,ОРГ1-02862,2,-1,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,...,352567.0,301268.0,288656.0,0.0,0.0,419767.0,468000.0,502917.0,409783.0,415297.0
548,ОРГ1-00471,2,-1,12.0,1.749719e+19,3917.0,3638.0,279.0,1088.0,2829.0,...,147035.0,75028.0,181244.0,793119.0,2279778.0,1819426.0,2434422.0,2644993.0,922389.0,2635880.0


In [64]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 550 entries, 0 to 549
Data columns (total 74 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            550 non-null    object 
 1   type                          550 non-null    int64  
 2   Вид образования               550 non-null    int8   
 3   Число опозданий               550 non-null    float64
 4   CallTime                      550 non-null    float64
 5   NumberOfCalls_worck           550 non-null    float64
 6   NumberOfCalls_free            550 non-null    float64
 7   NumberOfCallsfreesum          550 non-null    float64
 8   NumberOfCalls_x               550 non-null    float64
 9   NumberOfCalls_y               550 non-null    float64
 10  Время опоздания               550 non-null    float64
 11  filtime                       550 non-null    float64
 12  Специальность_x               550 non-null    float64
 13  Специ

plt.rcParams['figure.figsize']=(25,20)

corr = df_train.corr()
g = sns.heatmap(corr, square = True, annot=True)

plt.rcParams['figure.figsize']=(25,20)

corr = df_train.query('type==0').corr()
g = sns.heatmap(corr, square = True, annot=True)

plt.rcParams['figure.figsize']=(25,20)

corr = df_train.query('type==1').corr()
g = sns.heatmap(corr, square = True, annot=True)

plt.rcParams['figure.figsize']=(25,20)

corr = df_train.query('type==2').corr()
g = sns.heatmap(corr, square = True, annot=True)

plt.rcParams['figure.figsize']=(25,20)

corr = df_train.query('type==3').corr()
g = sns.heatmap(corr, square = True, annot=True)

## Выделим выборки

In [65]:
#df_train=df_train.query('type!=3')
#df_train=df_train.query('type!=1')
X = df_train.drop(['id', 'type',], axis = 1)
y = df_train["type"]#.map({0:0,1:0,2:0,3:1,})

# Стандартизация величин!

In [66]:
scaler = StandardScaler()
scaler.fit(X)
X=scaler.transform(X)

In [67]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

# smote

In [68]:
sm = SMOTE (random_state=0,k_neighbors=5)
X_train_res, y_train_res = sm.fit_resample (X_train,  y_train,) 
X_test_res, y_test_res = sm.fit_resample ( X_test,   y_test,) 

# Обучение модели

In [69]:
clf=OneVsRestClassifier(RandomForestClassifier(class_weight='balanced',criterion='entropy',max_depth= 4,min_samples_leaf= 13,min_samples_split= 8,n_estimators=100))
#clf=OneVsRestClassifier(LogisticRegression(max_iter=500,solver='newton-cg',penalty='l2',multi_class='ovr'))
#clf=LogisticRegression(max_iter=500,solver='newton-cg',penalty='l2',multi_class='ovr')

# Выбор фич для обучения

In [70]:
#selector = RFECV(clf, step=2, cv=15,n_jobs=-1)
#selector = selector.fit(X_train_res, y_train_res)
#selector.support_

In [71]:
#X_train_res=selector.transform(X_train_res)

In [72]:
X_train

array([[-0.44650572,  0.5005944 ,  0.36673743, ..., -0.10624493,
        -0.05034228, -0.00745386],
       [-0.44650572,  0.23395977,  1.39755398, ...,  1.56898187,
         1.15973429,  0.12763439],
       [-0.44650572, -0.45929027, -0.70999231, ..., -0.16653387,
        -0.03476146, -0.36458049],
       ...,
       [-0.74273991, -0.7259249 , -0.67253195, ...,  2.03250768,
         2.21771254,  1.9749183 ],
       [-0.44650572,  1.30049828,  0.57792453, ..., -0.47704296,
        -0.48218809, -0.32636268],
       [-0.44650572,  3.4691266 ,  1.10187449, ...,  1.46485236,
         0.61833904,  1.62632892]])

parametrs = {'criterion': ['gini', 'entropy', 'log_loss'],
             'class_weight':['balanced', 'balanced_subsample'], 
              'max_depth':range(1,40),
              'min_samples_leaf': range (2,15),
              'min_samples_split': range (2,15,) }
grid = GridSearchCV(RandomForestClassifier(), parametrs, scoring='recall_macro', cv=5,n_jobs=-1)
grid.fit(X_train, y_train)
grid.best_params_

gb_grid_params = {'learning_rate': [0.1, 0.05, 0.02, 0.01],
              'max_depth': [4, 6, 8, 10],
              'min_samples_leaf': [20, 50,100,150],
              #'max_features': [1.0, 0.3, 0.1] 
              }

gb_gs = GradientBoostingClassifier(n_estimators = 600)

clf = GridSearchCV(gb_gs,
                               gb_grid_params,
                               cv=2,
                               scoring='roc_auc',
                               verbose = 3, 
                               n_jobs=-1);
clf.fit(X_train, y_train)
clf.best_params_

for i in range(1,100):
    clf=OneVsRestClassifier(RandomForestClassifier(max_depth= 20, min_samples_leaf=1,min_samples_split=2,n_estimators=i))
    clf.fit(X_train, y_train)
    pred = clf.predict(X_test)
    print(i,"Recall score:", recall_score(y_test, pred, average='macro'))

In [73]:
clf.fit(X_train_res, y_train_res)

OneVsRestClassifier(estimator=RandomForestClassifier(class_weight='balanced',
                                                     criterion='entropy',
                                                     max_depth=4,
                                                     min_samples_leaf=13,
                                                     min_samples_split=8))

## Оценка точности

In [74]:
pred = clf.predict(X_test)

In [75]:
y_test.head(3)

195    2
79     2
480    0
Name: type, dtype: int64

In [76]:

print("Recall score:", recall_score(y_test, pred, average='macro'))

Recall score: 0.40128205128205124


In [77]:
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.43      0.62      0.51        24
           1       0.32      0.54      0.40        13
           2       0.69      0.28      0.39        40
           3       0.10      0.17      0.12         6

    accuracy                           0.41        83
   macro avg       0.38      0.40      0.36        83
weighted avg       0.51      0.41      0.41        83



## Генерация предсказания

In [78]:
df_test

,id,type
0,ОРГ1-02649,0
1,ОРГ2-05929,0
2,ОРГ2-05859,0
3,ОРГ1-00279,0
4,ОРГ2-01613,0
...,...,...
257,ОРГ1-02893,0
258,ОРГ2-01077,0
259,ОРГ1-00469,0
260,ОРГ1-02060,0


## Добавим фичи.

In [79]:
df_test = pd.merge(df_test, df_ed , on="id", how='left').drop_duplicates(subset="id")
df_test = df_test.drop(["Табельный номер руководителя", "Специальность"],axis = 1)
df_test["Вид образования"] = df_test["Вид образования"].astype('category')
df_test["Вид образования"] = df_test["Вид образования"].cat.codes
df_test = pd.merge(df_test, df_connection_time_res , on="id", how='left')#.drop(["event_type"],axis =1)

# Добавим суммарное время звоков

In [80]:
df_test = pd.merge(df_test, CallTime , on="id", how='left')

## Добавим колличество звонков в будни и выходные дни

In [81]:
df_test = pd.merge(df_test, NumberOfCalls , on="id", how='left')
df_test = pd.merge(df_test, NumberOfCallsW , on="id", how='left')
df_test = pd.merge(df_test, NumberOfCallsNW , on="id", how='left')
df_test = df_test.rename(columns={'NumberOfCalls_x': 'NumberOfCalls_worck','NumberOfCalls_y': 'NumberOfCalls_free','NumberOfCalls': 'NumberOfCallsfreesum'})

## Добавим колличество входящих и исходящих

In [82]:
df_test = pd.merge(df_test, NumberToUser , on="id", how='left')
df_test = pd.merge(df_test, NumberFromUser , on="id", how='left')

# Добавим суммарное время опоздания

In [83]:
df_test = pd.merge(df_test, falout , on="id", how='left')

# Добавим дельту прихода на работу

In [84]:
df_test  = pd.merge(df_test, filtime , on="id", how='left')

# Добавим профессию

In [85]:
profession=df_ed[['id','Специальность']].groupby('id')['Специальность'].min()
df_test = pd.merge(df_test, professionstart , on="id", how='left')
df_test = pd.merge(df_test, professionend , on="id", how='left')

In [86]:
df_test.head()

,id,type,Вид образования,Число опозданий,CallTime,NumberOfCalls_worck,NumberOfCalls_free,NumberOfCallsfreesum,NumberOfCalls_x,NumberOfCalls_y,Время опоздания,filtime,Специальность_x,Специальность_y
0,ОРГ1-02649,0,-1,NaN,1.045278e+17,4.0,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN
1,ОРГ2-05929,0,1,19.0,NaN,NaN,NaN,NaN,NaN,NaN,9.550722e+17,-1057.019584,999.0,999.0
2,ОРГ2-05859,0,8,104.0,NaN,NaN,NaN,NaN,NaN,NaN,5.190126e+18,-2103.577020,NaN,NaN
3,ОРГ1-00279,0,0,15.0,3.790214e+19,8163.0,8134.0,29.0,3679.0,4484.0,5.900416e+17,-570.718116,4.0,4.0
4,ОРГ2-01613,0,2,17.0,NaN,NaN,NaN,NaN,NaN,NaN,1.839148e+18,-386.013125,4.0,4.0


## Добавим колличество руководителей

In [87]:
df_test = pd.merge(df_test, boss , on="id", how='left')

# Добавим колличество завершенных заданий.

In [88]:
df_test = pd.merge(df_test,status_mission_count, on="id", how='left')
df_test = pd.merge(df_test,status_mission_count_in, on="id", how='left')
df_test = pd.merge(df_test,status_mission_count_nn, on="id", how='left')
df_test = pd.merge(df_test,status_mission_count_d, on="id", how='left')
df_test = pd.merge(df_test,status_mission_count_p, on="id", how='left')
df_test = pd.merge(df_test,status_mission_count_o, on="id", how='left')

# Добавим сумму просроченных дней

In [89]:
df_test = pd.merge(df_test,day_ful_mission, on="id", how='left')

# Добавим статус по просрочке с нарушением без науршения и их отношение.

In [90]:
df_test = pd.merge(df_test,fail_count_b, on="id", how='left')
df_test = pd.merge(df_test,fail_count_p, on="id", how='left')
df_test['bp']=df_test['статус_просрочек']/df_test['просрочено']

# Добавим общее время активности в сети.

In [91]:
df_test = pd.merge(df_test,timennetwork_time, on="id", how='left')

# Добавим среднее время простоя монитора 

In [92]:
df_test = pd.merge(df_test,monitor_stop, on="id", how='left')

# Добавим счетчик начала рабочего дня

In [93]:
df_test = pd.merge(df_test,start_day, on="id", how='left')

## Добавим колличество уникальных специальностей

In [94]:
df_test = pd.merge(df_test,prof_countis, on="id", how='left')

# Добавим оборотные виды документов

In [95]:
df_test = pd.merge(df_test,documents, on="id", how='left')

# Добавим месяцы звонков

In [96]:
df_test = pd.merge(df_test,monday_calls, on="id", how='left')

# Добавим месяцы в сети

In [97]:
df_test = pd.merge(df_test,monday_worck, on="id", how='left')

# Добавим месяцы активности за монитором

In [98]:
df_test = pd.merge(df_test,monday_Work, on="id", how='left')

# Добавим дельту дат звонка

In [99]:
df_test = pd.merge(df_test,last_cals, on="id", how='left')

# Добавим дельту сети

In [100]:
df_test = pd.merge(df_test,last_net, on="id", how='left')

# Добавим дельту работы за компьютером

In [101]:
df_test = pd.merge(df_test,last_worck, on="id", how='left')

# Звонки по месяцам

In [102]:
for i in range(1,13):
    calls_month_sum=pd.DataFrame(df_calls.query('monday==@i').groupby('id')['CallTime'].sum()).rename(columns={'CallTime': 'calls_month_sum'+ str(i)})
    df_test = pd.merge(df_test,calls_month_sum, on="id", how='left')
df_test

,id,type,Вид образования,Число опозданий,CallTime,NumberOfCalls_worck,NumberOfCalls_free,NumberOfCallsfreesum,NumberOfCalls_x,NumberOfCalls_y,...,calls_month_sum3,calls_month_sum4,calls_month_sum5,calls_month_sum6,calls_month_sum7,calls_month_sum8,calls_month_sum9,calls_month_sum10,calls_month_sum11,calls_month_sum12
0,ОРГ1-02649,0,-1,NaN,1.045278e+17,4.0,4.0,NaN,NaN,4.0,...,NaN,NaN,1.111111e+16,NaN,NaN,9.341667e+16,NaN,NaN,NaN,NaN
1,ОРГ2-05929,0,1,19.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ОРГ2-05859,0,8,104.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ОРГ1-00279,0,0,15.0,3.790214e+19,8163.0,8134.0,29.0,3679.0,4484.0,...,3.522008e+18,2.673262e+18,1.752227e+18,2.368776e+18,2.285503e+18,4.352574e+18,4.691294e+18,3.777449e+18,3.642086e+18,5.496340e+18
4,ОРГ2-01613,0,2,17.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,ОРГ1-02893,0,9,117.0,1.244414e+19,1404.0,1357.0,47.0,875.0,529.0,...,9.848444e+17,8.090306e+17,4.875556e+17,1.225361e+18,1.353189e+18,1.331778e+18,1.117333e+18,8.119306e+17,1.418531e+18,1.310083e+18
258,ОРГ2-01077,0,2,29.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
259,ОРГ1-00469,0,0,2.0,2.311854e+19,5103.0,5103.0,NaN,2646.0,2457.0,...,1.146544e+18,1.557994e+18,1.768487e+18,2.803575e+18,1.743061e+18,2.538170e+18,1.269558e+18,3.698911e+18,2.287967e+18,3.229044e+18
260,ОРГ1-02060,0,0,1.0,1.104076e+19,1251.0,1248.0,3.0,528.0,723.0,...,1.845894e+18,1.026917e+18,1.056889e+18,9.979444e+17,7.076389e+17,3.295722e+17,7.002472e+17,1.397314e+18,8.644444e+17,6.049444e+17


# Даты окончаний заданий по месяцам.

In [103]:
for i in range(1,13):
    tasks_month_sum=pd.DataFrame(df_tasks.query('month==@i').groupby('id')['Дата завершения задания фактическая'].count()).rename(columns={'Дата завершения задания фактическая': 'tasks_month_sum'+ str(i)})
    df_test = pd.merge(df_test,tasks_month_sum, on="id", how='left')
df_test

,id,type,Вид образования,Число опозданий,CallTime,NumberOfCalls_worck,NumberOfCalls_free,NumberOfCallsfreesum,NumberOfCalls_x,NumberOfCalls_y,...,tasks_month_sum3,tasks_month_sum4,tasks_month_sum5,tasks_month_sum6,tasks_month_sum7,tasks_month_sum8,tasks_month_sum9,tasks_month_sum10,tasks_month_sum11,tasks_month_sum12
0,ОРГ1-02649,0,-1,NaN,1.045278e+17,4.0,4.0,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ОРГ2-05929,0,1,19.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ОРГ2-05859,0,8,104.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ОРГ1-00279,0,0,15.0,3.790214e+19,8163.0,8134.0,29.0,3679.0,4484.0,...,43.0,28.0,26.0,20.0,42.0,32.0,38.0,75.0,74.0,77.0
4,ОРГ2-01613,0,2,17.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,ОРГ1-02893,0,9,117.0,1.244414e+19,1404.0,1357.0,47.0,875.0,529.0,...,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,2.0
258,ОРГ2-01077,0,2,29.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
259,ОРГ1-00469,0,0,2.0,2.311854e+19,5103.0,5103.0,NaN,2646.0,2457.0,...,13.0,11.0,3.0,33.0,34.0,76.0,15.0,14.0,9.0,1.0
260,ОРГ1-02060,0,0,1.0,1.104076e+19,1251.0,1248.0,3.0,528.0,723.0,...,5.0,3.0,1.0,NaN,1.0,1.0,NaN,1.0,28.0,NaN


# Добавим сеть по месяцам

In [104]:
for i in range(6,13):
    net_month_sum=pd.DataFrame(timennetwork.query('mondayW==@i').groupby('id')['monitor_Time'].sum()).rename(columns={'monitor_Time': 'net_month_sum'+ str(i)})
    df_test = pd.merge(df_test,net_month_sum, on="id", how='left')
df_test

,id,type,Вид образования,Число опозданий,CallTime,NumberOfCalls_worck,NumberOfCalls_free,NumberOfCallsfreesum,NumberOfCalls_x,NumberOfCalls_y,...,tasks_month_sum10,tasks_month_sum11,tasks_month_sum12,net_month_sum6,net_month_sum7,net_month_sum8,net_month_sum9,net_month_sum10,net_month_sum11,net_month_sum12
0,ОРГ1-02649,0,-1,NaN,1.045278e+17,4.0,4.0,NaN,NaN,4.0,...,NaN,NaN,NaN,184313.0,246367.0,544147.0,241275.0,NaN,NaN,NaN
1,ОРГ2-05929,0,1,19.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1201.0,300.0,265958.0,341855.0,347879.0,394428.0,108946.0
2,ОРГ2-05859,0,8,104.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,148527.0,477419.0,612585.0,917962.0,798497.0,503965.0,379073.0
3,ОРГ1-00279,0,0,15.0,3.790214e+19,8163.0,8134.0,29.0,3679.0,4484.0,...,75.0,74.0,77.0,279117.0,652295.0,616136.0,527806.0,446506.0,549430.0,667081.0
4,ОРГ2-01613,0,2,17.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,5.0,120645.0,323835.0,389523.0,346299.0,409943.0,292321.0,195703.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,ОРГ1-02893,0,9,117.0,1.244414e+19,1404.0,1357.0,47.0,875.0,529.0,...,NaN,NaN,2.0,NaN,NaN,213720.0,526847.0,507907.0,581635.0,582771.0
258,ОРГ2-01077,0,2,29.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.0,183345.0,422845.0,341796.0,298003.0,465470.0,557556.0,393093.0
259,ОРГ1-00469,0,0,2.0,2.311854e+19,5103.0,5103.0,NaN,2646.0,2457.0,...,14.0,9.0,1.0,263524.0,416008.0,720122.0,389946.0,585485.0,594240.0,672917.0
260,ОРГ1-02060,0,0,1.0,1.104076e+19,1251.0,1248.0,3.0,528.0,723.0,...,1.0,28.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80446.0


# Добавим работу за компьютером по месяцам

In [105]:
for i in range(6,13):
    worck_month_sum=pd.DataFrame(workingday.query('mondayWork==@i').groupby('id')['monitorTime'].sum()).rename(columns={'monitorTime': 'worck_month_sum'+ str(i)})
    df_test = pd.merge(df_test,worck_month_sum, on="id", how='left')
df_test

,id,type,Вид образования,Число опозданий,CallTime,NumberOfCalls_worck,NumberOfCalls_free,NumberOfCallsfreesum,NumberOfCalls_x,NumberOfCalls_y,...,net_month_sum10,net_month_sum11,net_month_sum12,worck_month_sum6,worck_month_sum7,worck_month_sum8,worck_month_sum9,worck_month_sum10,worck_month_sum11,worck_month_sum12
0,ОРГ1-02649,0,-1,NaN,1.045278e+17,4.0,4.0,NaN,NaN,4.0,...,NaN,NaN,NaN,879365.0,2631500.0,2329556.0,1418606.0,NaN,NaN,NaN
1,ОРГ2-05929,0,1,19.0,NaN,NaN,NaN,NaN,NaN,NaN,...,347879.0,394428.0,108946.0,291.0,62.0,947707.0,495221.0,367910.0,456080.0,143260.0
2,ОРГ2-05859,0,8,104.0,NaN,NaN,NaN,NaN,NaN,NaN,...,798497.0,503965.0,379073.0,254862.0,695597.0,670160.0,960461.0,783765.0,517915.0,398724.0
3,ОРГ1-00279,0,0,15.0,3.790214e+19,8163.0,8134.0,29.0,3679.0,4484.0,...,446506.0,549430.0,667081.0,481294.0,1931190.0,1395572.0,1495053.0,1215997.0,957291.0,2293596.0
4,ОРГ2-01613,0,2,17.0,NaN,NaN,NaN,NaN,NaN,NaN,...,409943.0,292321.0,195703.0,454835.0,1950484.0,1545721.0,1048113.0,1757733.0,634988.0,264763.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,ОРГ1-02893,0,9,117.0,1.244414e+19,1404.0,1357.0,47.0,875.0,529.0,...,507907.0,581635.0,582771.0,NaN,NaN,244616.0,558459.0,679791.0,583637.0,600593.0
258,ОРГ2-01077,0,2,29.0,NaN,NaN,NaN,NaN,NaN,NaN,...,465470.0,557556.0,393093.0,915312.0,2482764.0,2493516.0,1782930.0,2562410.0,2335598.0,2579047.0
259,ОРГ1-00469,0,0,2.0,2.311854e+19,5103.0,5103.0,NaN,2646.0,2457.0,...,585485.0,594240.0,672917.0,875603.0,2634942.0,2385401.0,1344973.0,2628958.0,2400127.0,2337736.0
260,ОРГ1-02060,0,0,1.0,1.104076e+19,1251.0,1248.0,3.0,528.0,723.0,...,NaN,NaN,80446.0,NaN,NaN,NaN,NaN,NaN,NaN,176382.0


# Удалим пустые значения

In [106]:
df_test = df_test.fillna(0)

In [107]:
df_test

,id,type,Вид образования,Число опозданий,CallTime,NumberOfCalls_worck,NumberOfCalls_free,NumberOfCallsfreesum,NumberOfCalls_x,NumberOfCalls_y,...,net_month_sum10,net_month_sum11,net_month_sum12,worck_month_sum6,worck_month_sum7,worck_month_sum8,worck_month_sum9,worck_month_sum10,worck_month_sum11,worck_month_sum12
0,ОРГ1-02649,0,-1,0.0,1.045278e+17,4.0,4.0,0.0,0.0,4.0,...,0.0,0.0,0.0,879365.0,2631500.0,2329556.0,1418606.0,0.0,0.0,0.0
1,ОРГ2-05929,0,1,19.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,...,347879.0,394428.0,108946.0,291.0,62.0,947707.0,495221.0,367910.0,456080.0,143260.0
2,ОРГ2-05859,0,8,104.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,...,798497.0,503965.0,379073.0,254862.0,695597.0,670160.0,960461.0,783765.0,517915.0,398724.0
3,ОРГ1-00279,0,0,15.0,3.790214e+19,8163.0,8134.0,29.0,3679.0,4484.0,...,446506.0,549430.0,667081.0,481294.0,1931190.0,1395572.0,1495053.0,1215997.0,957291.0,2293596.0
4,ОРГ2-01613,0,2,17.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,...,409943.0,292321.0,195703.0,454835.0,1950484.0,1545721.0,1048113.0,1757733.0,634988.0,264763.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,ОРГ1-02893,0,9,117.0,1.244414e+19,1404.0,1357.0,47.0,875.0,529.0,...,507907.0,581635.0,582771.0,0.0,0.0,244616.0,558459.0,679791.0,583637.0,600593.0
258,ОРГ2-01077,0,2,29.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,...,465470.0,557556.0,393093.0,915312.0,2482764.0,2493516.0,1782930.0,2562410.0,2335598.0,2579047.0
259,ОРГ1-00469,0,0,2.0,2.311854e+19,5103.0,5103.0,0.0,2646.0,2457.0,...,585485.0,594240.0,672917.0,875603.0,2634942.0,2385401.0,1344973.0,2628958.0,2400127.0,2337736.0
260,ОРГ1-02060,0,0,1.0,1.104076e+19,1251.0,1248.0,3.0,528.0,723.0,...,0.0,0.0,80446.0,0.0,0.0,0.0,0.0,0.0,0.0,176382.0


## Предсказания

In [108]:
df_test_pred = pd.read_csv("test.csv")
Xpred = df_test.drop(['id', 'type'], axis = 1)
Xpred=scaler.transform(Xpred)
#Xpred=selector.transform(Xpred)
preds = clf.predict(Xpred)
df_test_pred['type']=preds

In [109]:
df_test_pred.to_csv('43_52.csv', index=False)

In [110]:
df_test_pred['type'].value_counts()

2    109
0     69
1     51
3     33
Name: type, dtype: int64

In [111]:
c=pd.read_csv('43_52.csv')

In [112]:
c['type'].value_counts()

2    109
0     69
1     51
3     33
Name: type, dtype: int64